In [1]:

# import base64
# import requests
# import json

# CLIENT_ID = '9e4a6ae177824d17b3c9e87fcc3df72a'
# CLIENT_SECRET = 'ec500bc9877b47d89409d4f4cbcb8694'
# callback='http://google.com/callback/'

# def get_token():
#     auth_string=CLIENT_ID+":"+CLIENT_SECRET
#     auth_bytes=auth_string.encode('utf-8')
#     auth_base64=str(base64.b64encode(auth_bytes),'utf-8')

#     url="https://accounts.spotify.com/api/token"
#     headers={
#         'Authorization': 'Basic '+ auth_base64,
#         'Content-Type': 'application/x-www-form-urlencoded'
#     }

#     data={
#         'grant_type': 'client_credentials'
#     }

#     response=requests.post(url,headers=headers,data=data)
#     response_json=json.loads(response.content)
#     access_token=response_json["access_token"]

    
#     return access_token



# def get_auth_header(token):
#     return {"Authorization": "Bearer "+ token}


# token=get_token()
# print(token)

BQAAB8zIzUJ-rdR30OVMlgQqiSddyhC4HAXSs2nXEzj0PsbtOfuL-wMw51jot73pDF4DgrmyWZduCtioHFOr6WLgPHJTvtalcj_wsPqRAFS14M8Ynm4


In [173]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials,SpotifyOAuth
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

load_dotenv()

True

In [19]:
client_id=os.environ.get('CLIENT_ID')
client_secret=os.environ.get('CLIENT_SECRET')
redirect_uri="http://localhost"

In [25]:
scope ="user-library-read user-read-playback-state user-modify-playback-state"

client_cred_manager=SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
sp=spotipy.Spotify(client_credentials_manager=client_cred_manager,auth_manager=SpotifyOAuth(client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri,scope=scope))

In [190]:
results = sp.current_user_saved_tracks()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'], track['album'])

0 Flyleaf  –  All Around Me {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4IliztYDlfMvzQzbx50o60'}, 'href': 'https://api.spotify.com/v1/artists/4IliztYDlfMvzQzbx50o60', 'id': '4IliztYDlfMvzQzbx50o60', 'name': 'Flyleaf', 'type': 'artist', 'uri': 'spotify:artist:4IliztYDlfMvzQzbx50o60'}], 'available_markets': ['AR', 'AT', 'BE', 'BO', 'BR', 'BG', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'NL', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 'BT', 'BW', 'BF', 'CV', 'CW', 'DM', 'FJ', 'GM', 'GE', '

In [176]:
import time

def get_song_embedding(track_id,retries=5):
    try:

        features = sp.audio_features(track_id)
        if features:
            audio_features = features[0]
            #print(audio_features)
            embedding = [
                audio_features['danceability'],
                audio_features['energy'],
                audio_features['loudness'],
                audio_features['speechiness'],
                audio_features['acousticness'],
                audio_features['instrumentalness'],
                audio_features['liveness'],
                audio_features['valence'],
                audio_features['tempo']
            ]
            return np.array(embedding)
    except spotipy.SpotifyException as e:
        if e.http_status == 429:
            if retries > 0:
                retry_after = int(e.headers.get("Retry-After", 5))  # Get the Retry-After header
                print(f"Rate limit exceeded. Retrying in {retry_after} seconds...")
                time.sleep(retry_after)
                return get_song_embedding(track_id,retries-1)
            else:
                print("Max retries reached. Exiting.")
                return None
        else:
            raise e 

In [186]:
results = sp.current_user_saved_tracks()
track_id_to_name={}
new_song_embedding=[]
results = sp.current_user_saved_tracks(limit=50)
total_songs = results['total']
offset = 0
#print("User's Saved Tracks:")
while results:
    
    for i, item in enumerate(results['items']):
        track = item['track']
        track_id_to_name[offset + i + 1] = track['name']
        # Add your logic to get song embeddings here
        # new_song_embedding.append(get_song_embedding(track_id=track['id']))
    offset += len(results['items'])
    if len(results['items']) == 0 or offset >= total_songs:
        break
        #new_song_embedding.append(get_song_embedding(track_id=track['id']))
        #print(f"{track['name']} by {track['artists'][0]['name']} and {track['id']}")
    results = sp.current_user_saved_tracks(limit=50, offset=offset)


In [187]:
track_id_to_name

{1: 'All Around Me',
 2: 'Dopamine Detox',
 3: 'SUVs',
 4: 'Nothing But The Night',
 5: 'Be Strong',
 6: 'Poltergeist',
 7: 'MIDNIGHT',
 8: 'Darkness Is My Nickname',
 9: 'Pyres Of Varanasi',
 10: 'DO YOU DOUBT ME TRAITOR',
 11: 'Holloko - Afro Groove Mix',
 12: 'Everything I Wanted',
 13: 'Coming Back',
 14: 'When My Train Pulls In',
 15: 'Not Nice At All',
 16: "You're Gonna Go Far, Kid",
 17: 'Take Away My Pain',
 18: 'Black',
 19: 'The Answer Lies Within',
 20: 'Light In the Sky',
 21: 'Time Goes By',
 22: 'Feliteira',
 23: "Tell Me It's Over (feat. Summer Walker & 6LACK)",
 24: 'cellophane',
 25: 'Mumble Rapper vs. Lyricist',
 26: "Feelin' Myself",
 27: 'My Guy - Single Version',
 28: 'Like U',
 29: 'Falling Asleep At The Wheel',
 30: 'Green Eyes',
 31: 'striptease',
 32: 'Whispers, Pt. 1',
 33: 'Body',
 34: 'How to disappear',
 35: 'Bel Mercy',
 36: 'Devastación',
 37: 'Ratenplan',
 38: 'Concorde',
 39: 'Are You All Good?',
 40: "Don't Play",
 41: 'Codeine',
 42: 'Veridis Quo',
 

In [131]:
new_song_embedding=np.array(new_song_embedding)


In [132]:
new_song_embedding.shape

(611, 9)

In [134]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
# Assuming you have saved the scaler used during training
scaler = MinMaxScaler()
new_scaled_array = scaler.fit_transform(new_song_embedding)



In [135]:
new_scaled_array.shape

(611, 9)

In [138]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim),
            nn.ReLU()
        )
        self.decoder=nn.Sequential(
            nn.Linear(encoding_dim,input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        return encoded

# Load the model
input_dim = 9# Your input dimension
encoding_dim = 6# Your encoding dimension
model = Autoencoder(input_dim=input_dim, encoding_dim=encoding_dim)
model.load_state_dict(torch.load('spotify_dict_DL_model.pth'))
model.eval()

Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=9, out_features=6, bias=True)
    (1): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=6, out_features=9, bias=True)
    (1): Sigmoid()
  )
)

In [140]:

new_songs_features_tensor = torch.tensor(new_scaled_array, dtype=torch.float32)

with torch.no_grad():
    new_song_embeddings = model.encoder(new_songs_features_tensor).numpy()


In [159]:
new_song_embedding.shape

(611, 9)

In [142]:
model_song_embeddings=np.load('song_embeddings.npy')

In [160]:
model_song_embeddings.shape

(114000, 6)

In [161]:
import pandas as pd
df=pd.read_csv('songs_with_clusters.csv')
df.head(10)

song_index_to_name = df['track_name'].to_dict()


In [198]:
similarities = cosine_similarity(new_song_embeddings, model_song_embeddings)

# Generate playlist recommendations with unique songs
track_to_recommendation = {}
num_recommendations = 5
recommended_songs_set = set()

for i, seed_embedding in enumerate(new_song_embeddings):
    if i < similarities.shape[0]:  # Ensure the index is within bounds
        similar_indices = np.argsort(similarities[i])[::-1]
        #print(f"Recommendations for new song {i+1}: {track_id_to_name[i+1]}")
        
        recommend_list = []
        for idx in similar_indices:
            song_name = song_index_to_name[idx]
            if song_name not in recommended_songs_set:
                recommend_list.append(song_name)
                recommended_songs_set.add(song_name)
                if len(recommend_list) == num_recommendations:
                    break
        
        track_to_recommendation[track_id_to_name[i+1]] = recommend_list

# Display the recommendations
for track, recommendations in track_to_recommendation.items():
    print(f"\nRecommendations for {track}:")
    for rec in recommendations:
        print(f"  {rec}")



Recommendations for All Around Me:
  Style
  Iris
  Cuando salgo a tocar
  youthful days
  Stinker

Recommendations for Dopamine Detox:
  Cathedral Eyes
  Breathe
  Flip Ya Lid
  Fairytales
  Time to Make You Mine - Edit

Recommendations for SUVs:
  Idolatria
  Malokeiro Vencedor
  Aysel
  Your Obedient Servant
  P.L.F. (Pretty Little Freak)

Recommendations for Nothing But The Night:
  Ella Fue
  Mentime la Verdad
  A Novidade
  Blus Local
  Vuelve a Ti

Recommendations for Be Strong:
  Don't Quit - Original Mix
  Falling Apart
  In the Minds of Evil
  Durga
  Empathy

Recommendations for Poltergeist:
  C3PO
  Capsule in Space - Album Mix
  Protein Valve - Edit 1
  Journey To The Underworld
  Maybe Not - Rodriguez Jr. Remix

Recommendations for MIDNIGHT:
  Hyperpunk
  Off To Battle - Remix
  Pendulus
  Sign of the Times
  Song for Summer

Recommendations for Darkness Is My Nickname:
  Elimi Tut (feat. Ecem Telli)
  The Hound - Short Version
  Pullen Park
  I Cry - Ernest St Laurent R

In [200]:
import pickle 

with open('track_to_recommendation.pkl', 'wb') as file:
    pickle.dump(track_to_recommendation, file)



In [210]:
track_to_recommendation

{'All Around Me': ['Style',
  'Iris',
  'Cuando salgo a tocar',
  'youthful days',
  'Stinker'],
 'Dopamine Detox': ['Cathedral Eyes',
  'Breathe',
  'Flip Ya Lid',
  'Fairytales',
  'Time to Make You Mine - Edit'],
 'SUVs': ['Idolatria',
  'Malokeiro Vencedor',
  'Aysel',
  'Your Obedient Servant',
  'P.L.F. (Pretty Little Freak)'],
 'Nothing But The Night': ['Ella Fue',
  'Mentime la Verdad',
  'A Novidade',
  'Blus Local',
  'Vuelve a Ti'],
 'Be Strong': ["Don't Quit - Original Mix",
  'Falling Apart',
  'In the Minds of Evil',
  'Durga',
  'Empathy'],
 'Poltergeist': ['C3PO',
  'Capsule in Space - Album Mix',
  'Protein Valve - Edit 1',
  'Journey To The Underworld',
  'Maybe Not - Rodriguez Jr. Remix'],
 'MIDNIGHT': ['Hyperpunk',
  'Off To Battle - Remix',
  'Pendulus',
  'Sign of the Times',
  'Song for Summer'],
 'Darkness Is My Nickname': ['Elimi Tut (feat. Ecem Telli)',
  'The Hound - Short Version',
  'Pullen Park',
  'I Cry - Ernest St Laurent Remix',
  'Square Feet'],
 'Pyr

In [29]:
results=sp.current_user_playlists()

for idx, item in enumerate(results['items']):
    print(idx, item['name'],item['id'])

0 Why do you look so bluee? 05fiNclFkxsCiz3TkqbTdo
1 Pop that passes vibe check 5KrOpseWQeUwghIox1IyeO
2 Minimal 4L240eS515dFujpJraGN5L


In [70]:
playlist_id = '05fiNclFkxsCiz3TkqbTdo'
results = sp.playlist_tracks(playlist_id)
tracks = results['items']
for track in tracks:
    print(track['track']['id'])

5Ov7KyZfodsafPIn0hgGWT
1BQkVDlEOtYKOaotnJuNzz
6enELEFBmbFdCuS2cxrS8G
4pjo6uHUuP2FgoBfphagOd
193RPPANAhUa4KZmq35hUc
2sF93cCZuKgRLfWF6dNtfE


In [215]:
def get_album_art(track_name):
    try:
        results = sp.search(q='track:' + track_name, type='track', limit=1)
        if results['tracks']['items']:
            album_art_url = results['tracks']['items'][0]['album']['images'][0]['url']
            return album_art_url
    except spotipy.SpotifyException as e:
        print(f"Error getting album art for {track_name}: {e}")
    return None

In [ ]:
my_track_to_album_art={}
rec_track_to_album_art={}
for track_name in list(track_id_to_name.values()):
    album_art_url = get_album_art(track_name)
    my_track_to_album_art[track_name]=album_art_url
    rec_track=[]
    for x in track_to_recommendation[track_name]:
        album_art_url = get_album_art(track_name)
        rec_track_to_album_art[x]=album_art_url

In [216]:
rec_track_to_album_art={}
for track_name in list(track_id_to_name.values()):
    for x in track_to_recommendation[track_name]:
        album_art_url = get_album_art(x)
        rec_track_to_album_art[x]=album_art_url

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'track:Fascinados (feat. Joan Manuel Serrat, Leiva, Vetusta Morla, Iván Ferreiro, Loquillo, Zahara, Dani Martin, Albert Pla, Mikel (Izal), Noni (Lori Meyers), Santi Balmes, Xoel López, Anni B Sweet, Jeanette, Carlos Sadness, Nina (Morgan), Juan Alberto (Niños Mutantes), Miri Ros, Javiera Mena, Jorge Martí (La Habitación Roja), Rafa Val (Viva Suecia), Marc (Dorian), Alondra Bentley, Abraham Boba, Carlangas (Novedades Carminha), La Bien Querida, Martí Perarnau IV (Mucho), Nita (Fuel Fandango) & Shuarma (Elefantes))', 'limit': 1, 'offset': 0, 'type': 'track', 'market': None} returned 400 due to Bad request.


Error getting album art for Fascinados (feat. Joan Manuel Serrat, Leiva, Vetusta Morla, Iván Ferreiro, Loquillo, Zahara, Dani Martin, Albert Pla, Mikel (Izal), Noni (Lori Meyers), Santi Balmes, Xoel López, Anni B Sweet, Jeanette, Carlos Sadness, Nina (Morgan), Juan Alberto (Niños Mutantes), Miri Ros, Javiera Mena, Jorge Martí (La Habitación Roja), Rafa Val (Viva Suecia), Marc (Dorian), Alondra Bentley, Abraham Boba, Carlangas (Novedades Carminha), La Bien Querida, Martí Perarnau IV (Mucho), Nita (Fuel Fandango) & Shuarma (Elefantes)): http status: 400, code:-1 - https://api.spotify.com/v1/search?q=track%3AFascinados+%28feat.+Joan+Manuel+Serrat%2C+Leiva%2C+Vetusta+Morla%2C+Iv%C3%A1n+Ferreiro%2C+Loquillo%2C+Zahara%2C+Dani+Martin%2C+Albert+Pla%2C+Mikel+%28Izal%29%2C+Noni+%28Lori+Meyers%29%2C+Santi+Balmes%2C+Xoel+L%C3%B3pez%2C+Anni+B+Sweet%2C+Jeanette%2C+Carlos+Sadness%2C+Nina+%28Morgan%29%2C+Juan+Alberto+%28Ni%C3%B1os+Mutantes%29%2C+Miri+Ros%2C+Javiera+Mena%2C+Jorge+Mart%C3%AD+%28La+Habit

In [221]:
for x in track_to_recommendation['How to disappear']:
    print(x,rec_track_to_album_art[x])

3:00 AM https://i.scdn.co/image/ab67616d0000b273243ba5b923f5fd0b1ff5395f
Coffee https://i.scdn.co/image/ab67616d0000b2733cebcddece88bd433ff0ff43
Endless Love Mashup 2022 Lofi (Slowed & Reverb) None
Hey Nijame https://i.scdn.co/image/ab67616d0000b2734e7c92fbb72143e28ced4f97
Terra Dos Pássaros / Beijo Partido https://i.scdn.co/image/ab67616d0000b2730c98631e593543cabf4cf084


In [218]:
with open('my_track_to_album_art.pkl', 'wb') as file:
    pickle.dump(my_track_to_album_art, file)

In [219]:

with open('rec_track_to_album_art.pkl', 'wb') as file:
    pickle.dump(rec_track_to_album_art, file)